In [2]:
import json

In [2]:
data = {}
dateTable = {"第一天": 0, "第二天": 1, "第三天": 2, "第四天": 3, "第五天": 4, "第六天": 5}

with open('data/火车班次json数据', 'r', encoding = 'utf-8') as file:
    for line in file:
        currLine = json.loads(line)
        # print(currLine)
        currLineID = list(currLine["ticketInfo"].keys())[0]
        currLineStations = currLine["trainScheduleBody"]
        currLineMidSta = []
        for sta in currLineStations:
            if sta["content"][3] == "起点站":
                staTime = sta["content"][4].split(":")
                currLineMidSta.append((int(staTime[0]), int(staTime[1]), sta["content"][1]))
            else:
                staTime = sta["content"][3].split(":")
                currLineMidSta.append((int(staTime[0]) + 24 * dateTable[sta["content"][2]], int(staTime[1]), sta["content"][1]))
        currLineMidSta = sorted(currLineMidSta)
        for ind in range(len(currLineMidSta) - 1):
            if currLineMidSta[ind][2] not in data:
                data[currLineMidSta[ind][2]] = {}
            if currLineMidSta[ind + 1][2] not in data:
                data[currLineMidSta[ind + 1][2]] = {}
            if currLineMidSta[ind + 1][1] < currLineMidSta[ind][1]:
                currMinute = 60 - currLineMidSta[ind][1] + currLineMidSta[ind + 1][1]
                currHour = currLineMidSta[ind + 1][0] - currLineMidSta[ind][0] - 1
            else:
                currMinute = currLineMidSta[ind + 1][1] - currLineMidSta[ind][1]
                currHour = currLineMidSta[ind + 1][0] - currLineMidSta[ind][0]
            if currLineMidSta[ind + 1][2] in data[currLineMidSta[ind][2]]:
                if currHour > data[currLineMidSta[ind][2]][currLineMidSta[ind + 1][2]][0]:
                    continue
                elif currHour == data[currLineMidSta[ind][2]][currLineMidSta[ind + 1][2]][0]:
                    if currMinute >= data[currLineMidSta[ind][2]][currLineMidSta[ind + 1][2]][1]:
                        continue
            data[currLineMidSta[ind][2]][currLineMidSta[ind + 1][2]] = (currHour, currMinute)
            data[currLineMidSta[ind + 1][2]][currLineMidSta[ind][2]] = (currHour, currMinute)
        # break

for dataKey in data.keys():
    data[dataKey] = {k : v for k, v in sorted(data[dataKey].items(), key = lambda x : x[1])}

In [3]:
with open('processed_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)

In [4]:
with open('processed_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
cities = {}
for dat in data.keys():
    cities[dat] = dat[:2]

In [7]:
with open('sta_cities.json', 'w', encoding='utf-8') as f:
    json.dump(cities, f, ensure_ascii=False)

In [8]:
with open('sta_cities.json', 'r', encoding='utf-8') as f:
    cities = json.load(f)

In [10]:
cities_to_sta = {}
for sta, city in cities.items():
    if city not in cities_to_sta:
        cities_to_sta[city] = []
    cities_to_sta[city].append(sta)

In [13]:
cities_to_sta = {k: v for k, v in sorted(cities_to_sta.items(), key = lambda x: x[0])}

In [15]:
with open('cities_sta.json', 'w', encoding='utf-8') as f:
    json.dump(cities_to_sta, f, ensure_ascii=False)